# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tuktoyaktuk,69.4541,-133.0374,245.75,99,100,1.45,CA,1674367139
1,1,husavik,66.0449,-17.3389,272.85,76,91,6.22,IS,1674367139
2,2,biak,-0.9131,122.8766,301.39,66,100,0.70,ID,1674367139
3,3,puerto ayora,-0.7393,-90.3518,296.79,97,39,1.82,EC,1674367139
4,4,mandalgovi,45.7625,106.2708,251.91,74,100,9.40,MN,1674367139


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [47]:

# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ['City', 'Humidity']
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [49]:
# Narrow down cities that fit criteria and drop any results with null values
clean_cities_df = city_data_df.loc[(city_data_df["Max Temp"] < 300) & (city_data_df["Max Temp"] > 294) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)].dropna()

# Drop any rows with null values
clean_cities_df = clean_cities_df.dropna()

# Display sample data
clean_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
115,115,hilo,19.7297,-155.0900,297.75,88,0,0.89,US,1674367161
142,142,mecca,21.4267,39.8261,296.64,66,0,0.95,SA,1674367140
162,162,tura,25.5198,90.2201,297.42,26,0,2.18,IN,1674367169
336,336,maceio,-9.6658,-35.7353,295.84,94,0,1.54,BR,1674367207
368,368,ahipara,-35.1667,173.1667,294.90,73,0,3.33,NZ,1674367218


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tuktoyaktuk,CA,69.4541,-133.0374,99,
1,husavik,IS,66.0449,-17.3389,76,
2,biak,ID,-0.9131,122.8766,66,
3,puerto ayora,EC,-0.7393,-90.3518,97,
4,mandalgovi,MN,45.7625,106.2708,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "radius": radius,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame

    # YOUR CODE HERE
    lat = row["Lat"]
    lng = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    params["bias"] = f"proximity:{lat},{lng}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
  
    response = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tuktoyaktuk - nearest hotel: No hotel found
husavik - nearest hotel: No hotel found
biak - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
mandalgovi - nearest hotel: No hotel found
sorong - nearest hotel: No hotel found
belyy yar - nearest hotel: No hotel found
sao filipe - nearest hotel: No hotel found
abu dhabi - nearest hotel: No hotel found
nouadhibou - nearest hotel: No hotel found
shenzhen - nearest hotel: No hotel found
madera - nearest hotel: No hotel found
mar del plata - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
atuona - nearest hotel: No hotel found
mouila - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
wairoa - nearest hotel: No hotel found
pangody - nearest hotel: No hotel found
lavrentiya - nearest hotel: No hotel found
rikitea - nearest hotel: No hotel found
tiznit - nearest hotel: No hotel 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tuktoyaktuk,CA,69.4541,-133.0374,99,No hotel found
1,husavik,IS,66.0449,-17.3389,76,No hotel found
2,biak,ID,-0.9131,122.8766,66,No hotel found
3,puerto ayora,EC,-0.7393,-90.3518,97,No hotel found
4,mandalgovi,MN,45.7625,106.2708,74,No hotel found
...,...,...,...,...,...,...
574,les cayes,HT,18.2000,-73.7500,81,No hotel found
575,alice town,BS,25.7167,-79.3000,75,No hotel found
576,tazovskiy,RU,67.4667,78.7000,93,No hotel found
577,corvallis,US,44.5646,-123.2620,92,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [48]:


# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
"Lng",
"Lat",
geo = True,
tiles = "OSM",
frame_width = 600,
frame_height = 400,
size = "Humidity",
scale = 0.5,
color = "City",
hover_cols = ['City', 'Humidity','Country','Hotel Name']
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)